# DANAM Metadata Preparation for Transfer to HeidICON

This notebook is used along with the scripts clean_json and write_csv to query and analyze DANAM's image metadata quickly.
Queries is done via Pandas Dataframe and DataFrame can be checked using VSCode's variable viewer. 


In [1]:
import csv
import pandas as pd

from datetime import datetime
from scripts.clean_json import clean_json
from scripts import metadata_export as me

import warnings
warnings.filterwarnings("ignore")

## Read DANAM json export
Always replace with the latest export

In [2]:
# read DANAM json export
danam_export = "json\DANAM\Monument_2022-10-24_02-12-03.json"
danam_images = clean_json(danam_export)
danam_df = pd.DataFrame(danam_images)

Reset fix files and boolean values from last week

**WARNING: running this cell will delete everything in the fix files!**

In [3]:
ready_all,ready_images,ready_maps,ready_historical,ready_recent = me.reset(True)

## Preparing Metadata Uploads

In [4]:
year,month,date = (2022,5,1)

all,historical,images,maps,recent = me.prepare_metadata_from_mon(danam_df,
                    year,month,date,
                    ready_all,ready_maps,
                    ready_historical,
                    ready_images,ready_recent)

ALL: PLEASE CHECK USING VARIABLE VIEW
MAPS: PLEASE CHECK USING VARIABLE VIEW
HISTORICAL: PLEASE CHECK USING VARIABLE VIEW
ONLY PHOTOGRAPHS: PLEASE CHECK USING VARIABLE VIEW
RECENT CHANGES: PLEASE CHECK USING VARIABLE VIEW


In [5]:
ready_all = True
ready_images = True
ready_maps = True
ready_historical = True 
ready_recent = True

In [8]:
all,historical,images,maps,recent = me.prepare_metadata_from_mon(danam_df,
                    year,month,date,
                    ready_all,ready_maps,
                    ready_historical,
                    ready_images,ready_recent)

ALL: READY TO UPLOAD
MAPS: READY TO UPLOAD
HISTORICAL: READY TO UPLOAD
ONLY PHOTOGRAPHS: READY TO UPLOAD
RECENT CHANGES: READY TO UPLOAD


## Exporting Results to CSV for Weekly Metadata Transfer

In [10]:
# Writing to CSV 
cols = [
        'filename', 'caption', 'date1', 'date2', 'date', 'date3', 'agent', 'role', 'agent2', 'role2',
        'copyright', 'source', 'empty_column', 'notes', 'mon_id', 'class_code', 'classification', 'agent3', 
        'date_scan', 'license', 'url', 'rights_text', 'heidata', 'heidoc'
        ]

all_upload = pd.concat([all,historical,images,maps,recent])
all_upload.to_csv("csv/image_metadata_.csv", columns=cols, header=True, sep=';', index=False, quotechar = "\"", quoting=csv.QUOTE_ALL)
